In [1]:
import json
import numpy as np
import glob
import os

In [11]:
path = 'H:\\Dataset\\Ceymo\\test\\label_images_new\\'

final_dict = {}
for f in glob.glob(path + "*.json"):
    filename = os.path.basename(f).split('.')[0]
    #print(filename)
    with open(f, "rb") as f:
        data = json.load(f)
        
    new_list = []
    for shape in data['shapes']:
        label = shape['label']
        #print(label)
        all_points_x = []
        all_points_y = []
        for coord in shape['points']:
            x = coord[0]   ##圖片裁切
            y = coord[1]-760    #######################################
            all_points_x.append(x)
            all_points_y.append(y)
        all_points_x.append(all_points_x[0])
        all_points_y.append(all_points_y[0])

        info1 = {'name':label}
        info2 = {'name':'polyline','all_points_x':all_points_x, 'all_points_y':all_points_y}
        
        region = {'shape_attributes':info2, 'region_attributes': info1}
        
        new_list.append(region)
    #print(new_list)
    new = {filename+'.jpg':{'filename':filename+'.jpg','size':0, 'regions':new_list, 'file_attributes':{}}}
    final_dict.update(new)
#print(final_dict)

In [12]:
img_path = 'H:\\Dataset\\Ceymo\\test\\crop_images_new'
size_list = []
for image in os.listdir(img_path):
    size = os.path.getsize(os.path.join(img_path, image))
    size_list.append(size)
    #print(size)
new_dict2={}
for k, (keys, values) in enumerate(final_dict.items()): 
    new_keys = values['filename'] + str(size_list[k])
    new_dict2[keys] = new_keys
    
    ## replace file memory size
    values['size'] = size_list[k]
    
for old, new in new_dict2.items():
    final_dict[new] = final_dict.pop(old)
    
#print(final_dict)

In [13]:
file_name = 'labelme2via_label_images_new.json'
with open(file_name,'w') as outfile:
    json.dump(final_dict, outfile,ensure_ascii=False)